In [31]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora
import numpy as np
import ast
import os
from bs4 import BeautifulSoup
import re
import urllib.parse

In [69]:
df_relevance = pd.read_csv('datasets/relevent_snippets.csv')

In [22]:
df_relevance.head()

,Unnamed: 0,claim_id,snippet_content,snippet_date,snippet_id,snippet_pagenum,snippet_title,snippet_url,claim_content,claim_date,claim_label,claim_tag,claim_url,similarity_score
0,0,9,The swamp has declared war on ... comes to how...,17639,1414,6,Nancy Pelosi Archives - Empower Texans,https://empowertexans.com/tag/nancy-pelosi/,Most of our taxes are spent on the so-called ‘...,17645,False,— PolitiFact National,/truth-o-meter/statements/2018/apr/24/ro-khann...,0.738715
1,1,18,Trump Says North Korea Agreed to Denuclearize....,17639,2517,1,Navy Releases Service Details for Hero …,https://www.military.com/daily-news/2018/04/18...,Says North Korea has agreed to denuclearization.,17643,False,— PolitiFact National,/truth-o-meter/statements/2018/apr/22/donald-t...,0.662764
2,2,18,North Korea Says It Has ... North Korea has ag...,17640,2648,5,North Korea Says It Has Suspended …,http://newyork.cbslocal.com/2018/04/20/north-k...,Says North Korea has agreed to denuclearization.,17643,False,— PolitiFact National,/truth-o-meter/statements/2018/apr/22/donald-t...,0.669972
3,3,18,22/04/2018 · Regarder la vidéo · President Tru...,17642,2864,7,Trump says 'long way' from conclusion on …,http://www.dailymotion.com/video/x6i8wxk,Says North Korea has agreed to denuclearization.,17643,False,— PolitiFact National,/truth-o-meter/statements/2018/apr/22/donald-t...,0.833099
4,4,25,After Hezbollah gains in recent Lebanese elect...,17631,3889,0,Middle East News | The Jerusalem Post,https://www.jpost.com/Middle-East,Clinton Email: We must destroy Syria for Israel,17641,False,— PunditFact,/punditfact/statements/2018/apr/20/yournewswir...,0.693255


In [64]:
df_relevance_2 = df_relevance[['claim_id','claim_content','claim_url','snippet_id','snippet_pagenum','snippet_url', 'snippet_title', 'snippet_date', 'snippet_content', 'similarity_score']].sort_values(by=['similarity_score'],ascending=False).reset_index()
df_relevance_2['similarity_score']

0       1.000000
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000
6       1.000000
7       1.000000
8       1.000000
9       1.000000
10      1.000000
11      1.000000
12      1.000000
13      1.000000
14      1.000000
15      1.000000
16      1.000000
17      1.000000
18      1.000000
19      1.000000
20      1.000000
21      1.000000
22      1.000000
23      1.000000
24      1.000000
25      1.000000
26      1.000000
27      0.999999
28      0.998679
29      0.994926
          ...   
2161    0.653521
2162    0.653372
2163    0.653293
2164    0.653284
2165    0.653216
2166    0.653110
2167    0.653033
2168    0.653020
2169    0.652838
2170    0.652828
2171    0.652475
2172    0.652410
2173    0.652362
2174    0.652353
2175    0.652072
2176    0.652010
2177    0.651991
2178    0.651863
2179    0.651783
2180    0.651742
2181    0.651595
2182    0.651092
2183    0.650854
2184    0.650854
2185    0.650678
2186    0.650488
2187    0.650358
2188    0.6501

In [65]:
df2 = df_relevance_2[['claim_id', 'claim_content', 'claim_url', 'snippet_id','snippet_content','snippet_url','snippet_title', 'snippet_date','snippet_pagenum', 'similarity_score']]

In [66]:
len(df2)

2191

In [67]:
echantillon = {name:[] for name in ['claim_content','claim_url','snippet_title', 'snippet_content',\
                                    'snippet_url','snippet_pagenum', 'bing_url','similarity_score']}

for i in range(0,20):
    claim_content = df2['claim_content'][i]
    claim_url = df2['claim_url'][i]
    similarity_score = df2['similarity_score'][i]
    dt = df2['snippet_date'][i]
    bing_url = 'https://www.bing.com/search?q='+urllib.parse.quote(claim_content)+\
      '&filters=ex1%3a"ez5_'+str(dt)+'_'+str(dt+1)+'"'
    snippet_pagenum = df2['snippet_pagenum'][i]
    snippet_content = df2['snippet_content'][i]
    snippet_title = df2['snippet_title'][i]
    snippet_url = df2['snippet_url'][i]
    echantillon['claim_content'].append(claim_content)
    echantillon['claim_url'].append(claim_url)
    echantillon['snippet_title'].append(snippet_title)
    echantillon['snippet_content'].append(snippet_content)
    echantillon['snippet_url'].append(snippet_url)
    echantillon['snippet_pagenum'].append(snippet_pagenum)
    echantillon['bing_url'].append(bing_url)
    echantillon['similarity_score'].append(similarity_score)
    
print(echantillon)

{'claim_content': ["Two Iraqis came here to this country, were radicalized and they were the masterminds behind the Bowling Green massacre. … It didn't get covered", 'While on FAKE NEWS @CNN, Bernie Sanders was cut off for using the term fake news to describe the network. They said technical difficulties!', 'Only fools, or worse, are saying that our money losing Post Office makes money with Amazon. THEY LOSE A FORTUNE, and this will be changed.', 'Jackie Evancho’s album sales have skyrocketed after announcing her inauguration performance.', 'Only fools, or worse, are saying that our money losing Post Office makes money with Amazon. THEY LOSE A FORTUNE, and this will be changed.', 'I authorized Zero access to White House (actually turned him down many times) for author of phony book! I never spoke to him for book.', 'Obamacare is in a death spiral.', "\tEvery time we've cut the capital gains tax, the economy has grown. Whenever we raise the capital gains tax, it's been damaged.", 'Recen

In [70]:
df3 = pd.DataFrame(echantillon)
df3.head()

,bing_url,claim_content,claim_url,similarity_score,snippet_content,snippet_pagenum,snippet_title,snippet_url
0,https://www.bing.com/search?q=Two%20Iraqis%20c...,"Two Iraqis came here to this country, were rad...",/truth-o-meter/statements/2017/feb/03/kellyann...,1.0,... after two Iraqis came here to this country...,9,Internet Reacts to Kellyanne Conway’s …,http://washington.cbslocal.com/2017/02/03/inte...
1,https://www.bing.com/search?q=While%20on%20FAK...,"While on FAKE NEWS @CNN, Bernie Sanders was cu...",/truth-o-meter/statements/2017/feb/12/donald-t...,1.0,"While on FAKE NEWS @CNN, Bernie Sanders was cu...",2,12 | February | 2017 | News Corpse,https://www.newscorpse.com/ncWP/?m=20170212
2,https://www.bing.com/search?q=Only%20fools%2C%...,"Only fools, or worse, are saying that our mone...",/truth-o-meter/statements/2018/apr/02/donald-t...,1.0,"- Only fools, or worse, are saying that our mo...",6,"Trump on Twitter (April 2): DACA, NAFTA, …",https://www.metro.us/news/reuters/reuters-top-...
3,https://www.bing.com/search?q=Jackie%20Evancho...,Jackie Evancho’s album sales have skyrocketed ...,/pennsylvania/statements/2017/jan/05/donald-tr...,1.0,"... Adam Schiff, Nancy Giles, Randal Pinkett C...",2,"All in with Chris Hayes, Transcript …",http://www.msnbc.com/transcripts/all-in/2017-0...
4,https://www.bing.com/search?q=Only%20fools%2C%...,"Only fools, or worse, are saying that our mone...",/truth-o-meter/statements/2018/apr/02/donald-t...,1.0,"""Only fools, or worse, are saying that our mon...",1,Trump Triples Down In Amazon Feud: …,https://www.zerohedge.com/news/2018-03-31/satu...


In [71]:
df3.to_csv('pertinent_snippets.csv')

In [3]:
df_relevance.columns

Index(['Unnamed: 0', 'claim_id', 'snippet_content', 'snippet_date',
       'snippet_id', 'snippet_pagenum', 'snippet_title', 'snippet_url',
       'claim_content', 'claim_date', 'claim_label', 'claim_tag', 'claim_url',
       'similarity_score'],
      dtype='object')

In [4]:
documents = os.listdir('relevant_documents')

In [5]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True
#https://www.quora.com/How-can-I-extract-only-text-data-from-HTML-pages

In [6]:
def html_to_text(txt):
    soup = BeautifulSoup(txt,'html.parser')
    data = soup.findAll(text=True)
    res = list(filter(visible, data))
    return ' '.join(res)

In [7]:
dic_documents = {'snippet_id':[], 'document_content':[]}

for file_name in documents:
    snippet_id = int(file_name.split('.')[0])
    file = open('relevant_documents/'+file_name)
    txt = html_to_text(file.read())
    file.close()
    dic_documents['snippet_id'].append(snippet_id)
    dic_documents['document_content'].append(txt)
    
df_documents = pd.DataFrame(dic_documents)
df_documents.head(2)

,document_content,snippet_id
0,\n __ _ _ _ __| |_ (_)__ _____\n / _` | '_...,2620350
1,\n __ _ _ _ __| |_ (_)__ _____\n / _` | '_...,1357119


In [18]:
df_documents.to_csv('datasets/07_documents.csv')